In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import re

train = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\train.csv\train.csv")
test  = pd.read_csv(r"C:\Users\raven\Downloads\sf-crime\test.csv\test.csv")

train1 = train.drop(['Descript', 'Resolution'], axis = 1)

train1 = train1.query('Y != 90')

In [8]:
Dt = pd.to_datetime(train1['Dates'])
train1['Date'] = Dt.dt.date
train1['Year'] = Dt.dt.year
train1['Month'] = Dt.dt.month
train1['Day'] = Dt.dt.day
train1['Hour'] = Dt.dt.hour
train1.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
train1['DayOfWeek'] = le.fit_transform(train1['DayOfWeek'])

count_encoding = train1['PdDistrict'].value_counts().to_dict()
train1['PdDistrict'] = train1['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
train1['Address'] = train1['Address'].replace(Ad_dict,regex = True)


train1['Category'] = le.fit_transform(train1['Category'])

train1.head()
# # DayOfWeek...label, PdDistrict...count

,Category,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,37,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
1,21,6,105284,0,-122.425892,37.774599,2015-05-13,2015,5,13,23
2,21,6,105284,0,-122.424363,37.800414,2015-05-13,2015,5,13,23
3,16,6,105284,1,-122.426995,37.800873,2015-05-13,2015,5,13,23
4,16,6,49311,1,-122.438738,37.771541,2015-05-13,2015,5,13,23


In [9]:
Dt = pd.to_datetime(test['Dates'])
test['Date'] = Dt.dt.date
test['Year'] = Dt.dt.year
test['Month'] = Dt.dt.month
test['Day'] = Dt.dt.day
test['Hour'] = Dt.dt.hour
test.drop(["Dates"], axis = 1, inplace = True)

le = LabelEncoder()
test['DayOfWeek'] = le.fit_transform(test['DayOfWeek'])

count_encoding = test['PdDistrict'].value_counts().to_dict()
test['PdDistrict'] = test['PdDistrict'].map(count_encoding)

Ad_dict = {'.* / .*':0, '.*':1}
test['Address'] = test['Address'].replace(Ad_dict,regex = True)
test.head()

,Id,DayOfWeek,PdDistrict,Address,X,Y,Date,Year,Month,Day,Hour
0,0,3,89591,1,-122.399588,37.735051,2015-05-10,2015,5,10,23
1,1,3,89591,0,-122.391523,37.732432,2015-05-10,2015,5,10,23
2,2,3,107017,1,-122.426002,37.792212,2015-05-10,2015,5,10,23
3,3,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23
4,4,3,80084,1,-122.437394,37.721412,2015-05-10,2015,5,10,23


In [10]:
cols = ['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']

X = train1[cols]
Y = train1['Category']

X_test = test[cols]

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size = 0.3, random_state = 1219)

In [11]:
from catboost import CatBoostClassifier

def objective(trial):
    iterations=50
    depth=trial.suggest_int('depth', 6, 10)
    learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3)  
    random_strength=trial.suggest_int('random_strength', 0, 100)  
    bagging_temperature=trial.suggest_float('bagging_temperature', 0.5, 3.00) 
    od_type=trial.suggest_categorical('od_type', ['IncToDec', 'Iter'])
    od_wait=trial.suggest_int('od_wait', 10, 50)

    cb = CatBoostClassifier(iterations=iterations,
                            depth=depth,
                            learning_rate=learning_rate,
                            random_strength=random_strength,
                            bagging_temperature=bagging_temperature,
                            od_type=od_type,
                            od_wait=od_wait)
    
    score = cross_val_score(cb, X_train, y_train, cv=5, scoring="neg_log_loss")
    r2_mean = score.mean()
    print(r2_mean)

    return r2_mean

In [12]:
import optuna
study = optuna.create_study()
study.optimize(objective, n_trials=50, timeout=10800)

[I 2023-03-08 17:25:01,238] A new study created in memory with name: no-name-1d6196a4-0d1b-4f3c-b01e-63255e0dcc99


0:	learn: 3.5680528	total: 918ms	remaining: 45s
1:	learn: 3.4812168	total: 2.98s	remaining: 1m 11s
2:	learn: 3.4134079	total: 5.72s	remaining: 1m 29s
3:	learn: 3.3478538	total: 8.19s	remaining: 1m 34s
4:	learn: 3.2925437	total: 9.56s	remaining: 1m 26s
5:	learn: 3.2425526	total: 12s	remaining: 1m 27s
6:	learn: 3.2002324	total: 14s	remaining: 1m 26s
7:	learn: 3.1591135	total: 16.1s	remaining: 1m 24s
8:	learn: 3.1242136	total: 18.2s	remaining: 1m 22s
9:	learn: 3.0929951	total: 20.3s	remaining: 1m 21s
10:	learn: 3.0629123	total: 22.4s	remaining: 1m 19s
11:	learn: 3.0335085	total: 24.8s	remaining: 1m 18s
12:	learn: 3.0047097	total: 26.9s	remaining: 1m 16s
13:	learn: 2.9800059	total: 29s	remaining: 1m 14s
14:	learn: 2.9583465	total: 29.8s	remaining: 1m 9s
15:	learn: 2.9356913	total: 31.9s	remaining: 1m 7s
16:	learn: 2.9157788	total: 34.1s	remaining: 1m 6s
17:	learn: 2.8978734	total: 35.4s	remaining: 1m 2s
18:	learn: 2.8794399	total: 38.1s	remaining: 1m 2s
19:	learn: 2.8633102	total: 39.4s	re

[I 2023-03-08 17:33:21,324] Trial 0 finished with value: -2.6229452251956524 and parameters: {'depth': 7, 'learning_rate': 0.034889612514003236, 'random_strength': 100, 'bagging_temperature': 0.6071835524897075, 'od_type': 'IncToDec', 'od_wait': 18}. Best is trial 0 with value: -2.6229452251956524.


-2.6229452251956524
0:	learn: 3.0977684	total: 762ms	remaining: 37.3s
1:	learn: 2.9143214	total: 2.51s	remaining: 1m
2:	learn: 2.8096619	total: 7.37s	remaining: 1m 55s
3:	learn: 2.7370589	total: 8.63s	remaining: 1m 39s
4:	learn: 2.6809669	total: 13s	remaining: 1m 57s
5:	learn: 2.6374466	total: 17.7s	remaining: 2m 10s
6:	learn: 2.6166900	total: 19.6s	remaining: 2m
7:	learn: 2.6011243	total: 20.4s	remaining: 1m 47s
8:	learn: 2.5889390	total: 21.5s	remaining: 1m 37s
9:	learn: 2.5712355	total: 26.2s	remaining: 1m 44s
10:	learn: 2.5644990	total: 27.3s	remaining: 1m 36s
11:	learn: 2.5601308	total: 28.1s	remaining: 1m 28s
12:	learn: 2.5543671	total: 29.5s	remaining: 1m 23s
13:	learn: 2.5477788	total: 34s	remaining: 1m 27s
14:	learn: 2.5405457	total: 38.6s	remaining: 1m 30s
15:	learn: 2.5335622	total: 41s	remaining: 1m 27s
16:	learn: 2.5272157	total: 43.4s	remaining: 1m 24s
17:	learn: 2.5208471	total: 44.6s	remaining: 1m 19s
18:	learn: 2.5148963	total: 49.2s	remaining: 1m 20s
19:	learn: 2.5102

[I 2023-03-08 19:02:37,353] Trial 1 finished with value: -2.476477234963218 and parameters: {'depth': 8, 'learning_rate': 0.24579256382836476, 'random_strength': 91, 'bagging_temperature': 1.5635198792212956, 'od_type': 'IncToDec', 'od_wait': 10}. Best is trial 0 with value: -2.6229452251956524.


...

-2.4920656722258148
0:	learn: 3.0385534	total: 817ms	remaining: 40s
1:	learn: 2.8584118	total: 2.41s	remaining: 57.9s
2:	learn: 2.7521839	total: 4.05s	remaining: 1m 3s
3:	learn: 2.6886845	total: 5.62s	remaining: 1m 4s
4:	learn: 2.6449371	total: 7.25s	remaining: 1m 5s
5:	learn: 2.6065053	total: 8.9s	remaining: 1m 5s
6:	learn: 2.5764383	total: 10.5s	remaining: 1m 4s
7:	learn: 2.5597432	total: 12.2s	remaining: 1m 3s
8:	learn: 2.5415898	total: 13.8s	remaining: 1m 2s
9:	learn: 2.5323671	total: 15.3s	remaining: 1m 1s
10:	learn: 2.5187176	total: 17s	remaining: 1m
11:	learn: 2.5160056	total: 17.9s	remaining: 56.8s
12:	learn: 2.5111864	total: 19.6s	remaining: 55.7s
13:	learn: 2.5090175	total: 20.7s	remaining: 53.3s
14:	learn: 2.5032661	total: 22.4s	remaining: 52.2s
15:	learn: 2.5007700	total: 24s	remaining: 51.1s
16:	learn: 2.4956398	total: 25.7s	remaining: 49.9s
17:	learn: 2.4928965	total: 27.3s	remaining: 48.6s
18:	learn: 2.4903493	total: 29s	remaining: 47.3s
19:	learn: 2.4859176	total: 30.6s

[I 2023-03-08 19:16:53,104] Trial 3 finished with value: -2.4656272292948422 and parameters: {'depth': 6, 'learning_rate': 0.2852728795999983, 'random_strength': 20, 'bagging_temperature': 2.252697540678127, 'od_type': 'Iter', 'od_wait': 23}. Best is trial 0 with value: -2.6229452251956524.


-2.4656272292948422
0:	learn: 3.2909672	total: 491ms	remaining: 24.1s
1:	learn: 3.1087965	total: 1.4s	remaining: 33.5s
2:	learn: 2.9892436	total: 5.59s	remaining: 1m 27s
3:	learn: 2.8966516	total: 13.7s	remaining: 2m 37s
4:	learn: 2.8181366	total: 20.6s	remaining: 3m 5s
5:	learn: 2.7663592	total: 28s	remaining: 3m 25s
6:	learn: 2.7224583	total: 35.1s	remaining: 3m 35s
7:	learn: 2.6861041	total: 42.3s	remaining: 3m 42s
8:	learn: 2.6613918	total: 42.9s	remaining: 3m 15s
9:	learn: 2.6393984	total: 50s	remaining: 3m 20s
10:	learn: 2.6156316	total: 57.1s	remaining: 3m 22s
11:	learn: 2.5912437	total: 1m 4s	remaining: 3m 22s
12:	learn: 2.5793393	total: 1m 4s	remaining: 3m 3s
13:	learn: 2.5638318	total: 1m 11s	remaining: 3m 4s
14:	learn: 2.5481425	total: 1m 18s	remaining: 3m 3s
15:	learn: 2.5375898	total: 1m 25s	remaining: 3m 2s
16:	learn: 2.5268780	total: 1m 32s	remaining: 3m
17:	learn: 2.5204395	total: 1m 36s	remaining: 2m 51s
18:	learn: 2.5168816	total: 1m 37s	remaining: 2m 38s
19:	learn: 2

[I 2023-03-08 21:19:41,704] Trial 4 finished with value: -2.468631163402579 and parameters: {'depth': 9, 'learning_rate': 0.1468835731636316, 'random_strength': 46, 'bagging_temperature': 2.7963013094401945, 'od_type': 'IncToDec', 'od_wait': 12}. Best is trial 0 with value: -2.6229452251956524.


-2.468631163402579


In [13]:
optimised_cb = CatBoostClassifier(iterations=50,
                            depth=study.best_params['depth'],
                            learning_rate=study.best_params['learning_rate'],
                            random_strength=study.best_params['random_strength'],
                            bagging_temperature=study.best_params['bagging_temperature'],
                            od_type=study.best_params['od_type'],
                            od_wait=study.best_params['od_wait'])
optimised_cb.fit(X_train ,y_train, early_stopping_rounds=10)

0:	learn: 3.5680445	total: 598ms	remaining: 29.3s
1:	learn: 3.4810116	total: 1.92s	remaining: 46.2s
2:	learn: 3.4131164	total: 3.89s	remaining: 1m
3:	learn: 3.3474690	total: 5.86s	remaining: 1m 7s
4:	learn: 3.2921936	total: 6.82s	remaining: 1m 1s
5:	learn: 3.2421182	total: 8.8s	remaining: 1m 4s
6:	learn: 3.1998115	total: 10.9s	remaining: 1m 6s
7:	learn: 3.1586767	total: 12.8s	remaining: 1m 7s
8:	learn: 3.1237543	total: 14.8s	remaining: 1m 7s
9:	learn: 3.0924538	total: 16.8s	remaining: 1m 7s
10:	learn: 3.0623566	total: 18.8s	remaining: 1m 6s
11:	learn: 3.0330135	total: 20.8s	remaining: 1m 5s
12:	learn: 3.0042589	total: 22.8s	remaining: 1m 4s
13:	learn: 2.9795593	total: 24.8s	remaining: 1m 3s
14:	learn: 2.9579389	total: 25.4s	remaining: 59.2s
15:	learn: 2.9353874	total: 27.3s	remaining: 57.9s
16:	learn: 2.9154633	total: 29.3s	remaining: 56.8s
17:	learn: 2.8975639	total: 30.2s	remaining: 53.8s
18:	learn: 2.8791204	total: 32.3s	remaining: 52.7s
19:	learn: 2.8630181	total: 33.3s	remaining: 

In [14]:
y_pred = optimised_cb.predict(X_val)

In [15]:
accuracy_score(y_val, y_pred)

0.22573701095313123

In [16]:
cb_submission = optimised_cb.predict(X_test)

In [17]:
import csv
pred = optimised_cb.predict_proba(test[['DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y', 'Year', 'Month', 'Day', 'Hour']])
submission = pd.DataFrame(columns=['Id'], data=test)
submission = pd.concat([submission, pd.DataFrame(pred, columns=["ARSON", "ASSAULT", "BAD CHECKS", "BRIBERY", "BURGLARY", "DISORDERLY CONDUCT",
"DRIVING UNDER THE INFLUENCE", "DRUG/NARCOTIC", "DRUNKENNESS", "EMBEZZLEMENT", "EXTORTION",
"FAMILY OFFENSES", "FORGERY/COUNTERFEITING", "FRAUD", "GAMBLING", "KIDNAPPING", "LARCENY/THEFT",
"LIQUOR LAWS", "LOITERING", "MISSING PERSON", "NON-CRIMINAL", "OTHER OFFENSES", "PORNOGRAPHY/OBSCENE MAT",
"PROSTITUTION", "RECOVERED VEHICLE", "ROBBERY", "RUNAWAY", "SECONDARY CODES", "SEX OFFENSES FORCIBLE",
"SEX OFFENSES NON FORCIBLE", "STOLEN PROPERTY", "SUICIDE", "SUSPICIOUS OCC", "TREA", "TRESPASS",
"VANDALISM", "VEHICLE THEFT", "WARRANTS", "WEAPON LAWS"])], axis=1)

In [ ]:
submission.to_csv("cat1opt.csv", index=False)